[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sapiosciences/sapio-py-tutorials/blob/master/8_deepchem_solubility.ipynb)

In [ ]:
!pip install --pre deepchem

Looking in indexes: https://py.sapiosciences.com/superjenkins/dev
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: sapiopylib
    Found existing installation: sapiopylib 0.1
    Uninstalling sapiopylib-0.1:
      Successfully uninstalled sapiopylib-0.1


[DeepChem](https://github.com/deepchem/deepchem/) is a Chemistry library that runs on top of PyTorch and Tensorflow.

You can use DeepChem to build AI models. In this example, we will be using their tutorial example to work with our system to determine soluability of registered compounds in Sapio.

In [ ]:
import deepchem as dc
dc.__version__
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)

2022-10-17 16:29:07.278060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-17 16:29:07.278261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-17 16:29:07.278330: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-17 16:29:07.278396: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-17 16:29:07.278463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [ ]:
from pandas import DataFrame
from sapiopylib.rest.User import SapioUser
from sapiopylib.rest.DataMgmtService import DataMgmtServer

user = SapioUser(url="https://linux-vm:8443/webservice/api",
                 guid="3c232543-f407-4828-aae5-b33d4cd31fa7", account_name="sapio",
                 username="pyRestTest", password="Password1!", verify_ssl_cert=False)
data_record_manager = DataMgmtServer.get_data_record_manager(user)

compound_parts_page = data_record_manager.query_all_records_of_type('CompoundPart')
compound_parts = compound_parts_page.result_list

compound_df: DataFrame = data_record_manager.get_data_frame(compound_parts)
display(compound_df)

,ExactMass,PartNumber,ConsumableType,SoluabilityEstimate,DateCreated,SubstanceClass,Density,QuantityPerItem,Formula,PolarSurfaceArea,...,CreatedBy,Charge,SMILES,MOL,QuantityOnHand,Units,TotalHBondCount,DataRecordName,VeloxLastModifiedDate,molregno
0,693.23042,Compound,Egfr-IN-7,None,1666015301302,,,None,C32H41BrN9O2P,111.64,...,yqiao,0,COc1cc(N2CCC(N3CCN(C)CC3)CC2)c(C)cc1Nc1ncc(Br)...,\n RDKit 2D\n\n 45 50 0 0 0 0...,0.0,,2,Egfr-IN-7,1666015624644,None


In [ ]:
import math
from deepchem.data import NumpyDataset
from deepchem.feat import ConvMolFeaturizer

smiles = compound_df.loc[:, 'SMILES']
featurizer = ConvMolFeaturizer()
mols = featurizer.featurize(smiles)

sapio_dataset = NumpyDataset(X=mols, ids=smiles)
predictions = model.predict(sapio_dataset, transformers=transformers)

#Note: the model is log(soluability). We will need to use a power function to invert.
for prediction in predictions:
    print (math.pow(10, prediction))

0.008315065200545577
